In [1]:
using Query
using DataFrames
using CSVFiles
using Glob

INFO: Recompiling stale cache file /home/wheel/oxinabox/.julia/lib/v0.6/DataFrames.ji for module DataFrames.


In [2]:
2+34

36

In [3]:

dist_df = DataFrame(load("results/raw/dist_full.csv" ))



for filename in ["ord.csv", "full.csv", "extrapo.csv"]
    df = DataFrame(load(joinpath("results", "raw", "dist_"*filename)))

    res_form = @from ii in df begin
        @select {method="Distribution Mean " .* ii.method, mse=ii.mse_to_distmean}
    end

    save(joinpath("results", "distpoint_"*filename), res_form)
end
###############################


In [4]:
Base.String() = ""
Base.Float64() = NaN



method_name(method)::String = join(
    filter(x->x ∉ ["Extrapolating", "Non-extrapolating"],
           split(method)), " ")





extrapo_res(df) = df |> @filter(contains(_.method, "Extrapolating"))
nonextrapo_res(df) = df |> @filter(contains(_.method, "Non-extrapolating"))

###########################
filename =  "results/raw/dist_extrapo.csv" 
df = DataFrame(load(filename))

res_form = @from nonext in nonextrapo_res(df) begin
    @let base_method = method_name(nonext.method)
    @left_outer_join ext in extrapo_res(df) on base_method equals method_name(ext.method)
    @select {method=base_method, nonextrapolatingperp=nonext.perp, extrapolatingperp = ext.perp}
end

save("results/res_dist_extrapo.csv", res_form)
###############################


In [5]:

###########################

point_extrapos = map(["raw/point_extrapo.csv", "distpoint_extrapo.csv"]) do filename
    df = DataFrame(load(joinpath("results", filename)))

    res_form = @from nonext in nonextrapo_res(df) begin
        @let base_method = method_name(nonext.method)
        @left_outer_join ext in extrapo_res(df) on base_method equals method_name(ext.method)
        @select {method=base_method,
            nonextrapolatingmse=nonext.mse,
            extrapolatingmse= ext.mse}
        @collect DataFrame
    end
end
###############################

save("results/res_point_comb_extrapo.csv", vcat(point_extrapos...))

In [6]:

save("results/res_point_comb_full.csv",
    vcat(DataFrame.(load.(["results/raw/point_full.csv", "results/distpoint_full.csv"]))...)
    )
save("results/res_point_comb_ord.csv",
    vcat(DataFrame.(load.(["results/raw/point_ord.csv", "results/distpoint_ord.csv"]))...)
    )


In [7]:
cp("results/raw/dist_full.csv","results/res_dist_full.csv", remove_destination=true)
cp("results/raw/dist_ord.csv","results/res_dist_ord.csv", remove_destination=true)

In [8]:
1

1

In [9]:
## avoid case-senstive names

In [10]:
for (path,dirs,files) in walkdir("demo")
    for fn in files
        newfn = replace(fn, r"[A-Z]", s"1\0")
        if fn != newfn && '1' ∉ fn
            mv(joinpath(path,fn), joinpath(path,newfn))
        end
    end
end

In [11]:
fn =  glob(glob"results/*dist*.csv") |> last
    df = DataFrame(load(fn))

,method,perp,mse_to_distmean
1,Direct,4.08852e6,0.0648013
2,Direct-smoothed,8.93602e5,0.0648423
3,SOWE,9.1701e5,0.0656019
4,SOWE-smoothed,9.16069e5,0.0657174
5,CNN,9.55151e5,0.0658269
6,CNN-smoothed,9.79344e5,0.0658741
7,RNN,2.08376e6,0.0953645
8,RNN-smoothed,2.04503e6,0.0966241


# Drop Smoothed results

In [35]:
function retitle_direct!(methods, fn)
    methods[:] = map(methods) do x
        if r".*dist.*"(fn)
            x = replace(x, "Direct-smoothed", "Operational Upper Bound")
        elseif r".*point.*"(fn)
            x= replace(x, "Direct", "Operational Upper Bound")
        end
        x
    end
    methods
end

function slice(func, dest)
    mkpath(dest)
    for fn in glob(glob"results/*.csv")
        @show fn
        df = DataFrame(load(fn))
        retitle_direct!(df[:method], fn)
        df = func(df)
        @show df
        outfn = joinpath(dest, basename(fn))
        save(outfn, df, missingstring=NaN)
    end
end

################
slice("results/regular") do df
    df = @from row in df begin
        @where !contains(row.method, "smoothed") && !contains(row.method, "Direct")
        @select row
        @collect DataFrame
    end   
end
slice("results/smoothed") do df
    df = @from row in df begin
        @where contains(row.method, "smoothed") && !contains(row.method, "Direct")
        @select row
        @collect DataFrame
    end
end

fn = "results/distpoint_extrapo.csv"
df = 7×2 DataFrames.DataFrame
│ Row │ method                                                      │ mse       │
├─────┼─────────────────────────────────────────────────────────────┼───────────┤
│ 1   │ Distribution Mean Operational Upper Bound Non-extrapolating │ 0.0621801 │
│ 2   │ Distribution Mean SOWE Extrapolating                        │ 0.0763948 │
│ 3   │ Distribution Mean SOWE Non-extrapolating                    │ 0.0728559 │
│ 4   │ Distribution Mean CNN Extrapolating                         │ 0.0843781 │
│ 5   │ Distribution Mean CNN Non-extrapolating                     │ 0.0729186 │
│ 6   │ Distribution Mean RNN Extrapolating                         │ 0.15215   │
│ 7   │ Distribution Mean RNN Non-extrapolating                     │ 0.10479   │
fn = "results/distpoint_full.csv"
df = 4×2 DataFrames.DataFrame
│ Row │ method                                    │ mse       │
├─────┼───────────────────────────────────────────┼───────────┤
│ 1

In [36]:

1

1

In [13]:
@which save("", df)

save(s::Union{AbstractString, IO}, args...) in FileIO at /home/wheel/oxinabox/.julia/v0.6/FileIO/src/loadsave.jl:113

In [14]:
@which save(FileIO.query("foo.csv"), df)

save(q::FileIO.Formatted{F}, data...) where F in FileIO at /home/wheel/oxinabox/.julia/v0.6/FileIO/src/loadsave.jl:194